In [1]:
from __future__ import print_function, division
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import requests
import json
from io import StringIO
import io
import shapely

DEVELOPE = True

#### weather data - Local Climatological Data (LCD)

In [2]:
df = pd.read_csv('weather130101-171130.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10,11,14,15,20,23,25,40,44,45,67,68,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print ("Shape of data\n{}".format(df.shape))

Shape of data
(55996, 90)


In [4]:
df.drop(['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 
              'LONGITUDE', 'REPORTTPYE', 'HOURLYSKYCONDITIONS', 
              'HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE', 
              'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 
              'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 
              'HOURLYDewPointTempF', 'HOURLYDewPointTempC', 
              'HOURLYRelativeHumidity', 'HOURLYWindDirection',
              'HOURLYPrecip', 'MonthlyMaximumTemp',
       'MonthlyMinimumTemp', 'MonthlyMeanTemp', 'MonthlyAverageRH',
       'MonthlyDewpointTemp', 'MonthlyWetBulbTemp',
       'MonthlyAvgHeatingDegreeDays', 'MonthlyAvgCoolingDegreeDays',
       'MonthlyStationPressure', 'MonthlySeaLevelPressure',
       'MonthlyAverageWindSpeed', 'MonthlyTotalSnowfall',
       'MonthlyDeptFromNormalMaximumTemp', 'MonthlyDeptFromNormalMinimumTemp',
       'MonthlyDeptFromNormalAverageTemp', 'MonthlyDeptFromNormalPrecip',
       'MonthlyTotalLiquidPrecip', 'MonthlyGreatestPrecip',
       'MonthlyGreatestPrecipDate', 'MonthlyGreatestSnowfall',
       'MonthlyGreatestSnowfallDate', 'MonthlyGreatestSnowDepth',
       'MonthlyGreatestSnowDepthDate', 'MonthlyDaysWithGT90Temp',
       'MonthlyDaysWithLT32Temp', 'MonthlyDaysWithGT32Temp',
       'MonthlyDaysWithLT0Temp', 'MonthlyDaysWithGT001Precip',
       'MonthlyDaysWithGT010Precip', 'MonthlyDaysWithGT1Snow',
       'MonthlyMaxSeaLevelPressureValue', 'MonthlyMaxSeaLevelPressureDate',
       'MonthlyMaxSeaLevelPressureTime', 'MonthlyMinSeaLevelPressureValue',
       'MonthlyMinSeaLevelPressureDate', 'MonthlyMinSeaLevelPressureTime',
       'MonthlyTotalHeatingDegreeDays', 'MonthlyTotalCoolingDegreeDays',
       'MonthlyDeptFromNormalHeatingDD', 'MonthlyDeptFromNormalCoolingDD',
       'MonthlyTotalSeasonToDateHeatingDD',
       'MonthlyTotalSeasonToDateCoolingDD','DAILYAverageStationPressure',
       'DAILYAverageSeaLevelPressure', 'DAILYAverageWindSpeed',
       'DAILYPeakWindSpeed', 'PeakWindDirection', 'DAILYSustainedWindSpeed',
       'DAILYSustainedWindDirection','DAILYMaximumDryBulbTemp',
       'DAILYMinimumDryBulbTemp', 'DAILYAverageDryBulbTemp',
       'DAILYDeptFromNormalAverageTemp', 'DAILYAverageRelativeHumidity',
       'DAILYAverageDewPointTemp', 'DAILYAverageWetBulbTemp',
       'DAILYHeatingDegreeDays', 'DAILYCoolingDegreeDays'], axis=1, inplace=True)

In [5]:
df.columns

Index(['DATE', 'HOURLYWindSpeed', 'HOURLYWindGustSpeed',
       'HOURLYStationPressure', 'HOURLYPressureTendency',
       'HOURLYPressureChange', 'HOURLYSeaLevelPressure',
       'HOURLYAltimeterSetting', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather',
       'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth'],
      dtype='object')

In [6]:
df.count()

DATE                      55996
HOURLYWindSpeed           50749
HOURLYWindGustSpeed        9362
HOURLYStationPressure     53604
HOURLYPressureTendency    14083
HOURLYPressureChange       2423
HOURLYSeaLevelPressure    42318
HOURLYAltimeterSetting    53660
DAILYSunrise              55996
DAILYSunset               55996
DAILYWeather                776
DAILYPrecip                2388
DAILYSnowfall              1794
DAILYSnowDepth             1794
dtype: int64

In [7]:
df.head(1)

,DATE,HOURLYWindSpeed,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYAltimeterSetting,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth
0,2013-01-01 00:51,10.0,22.0,29.72,8.0,NaN,29.87,29.89,720,1639,NaN,NaN,NaN,NaN


DZ:01 - Drizzle | RA:02 - Rain | SN:03 - Snow | SG:04 - Snow Grains | IC:05 - Ice Crystals 
PL:06 - Ice Pellets | GR:07 - Hail | GS:08- Small Hail an/or Snow Pellets 
UP:09 - Unknown Precipitation | BR:1 - Mist | FG:2 - Fog | FU:3 - Smoke
VA:4 - Volcanic Ash | DU:5 - Widespread Dust | SA:6 - Sand | HZ:7 - Haze
PY:8 - Spray | PO:1 - Well developed dust/sand whirls | SQ:2 - Squalls
FC:3 - Funnel CLoud, Waterspout or Tornado | SS:4 - Sandstorm | DS:5 - Duststorm

In [8]:
df1 = df[['DATE', 'HOURLYWindSpeed', 'HOURLYWindGustSpeed',
       'HOURLYStationPressure', 'DAILYSunrise', 'DAILYSunset',
       'DAILYPrecip', 'DAILYSnowfall']]
df1.head(1)

,DATE,HOURLYWindSpeed,HOURLYWindGustSpeed,HOURLYStationPressure,DAILYSunrise,DAILYSunset,DAILYPrecip,DAILYSnowfall
0,2013-01-01 00:51,10.0,22.0,29.72,720,1639,NaN,NaN


#### weather data -- daily summary

In [9]:
df2 = pd.read_csv('weather_daily_summary_2013_now.csv')

In [10]:
print ("Shape of data\n{}".format(df2.shape))

Shape of data
(1794, 27)


In [11]:
df2.count()

STATION      1794
NAME         1794
LATITUDE     1794
LONGITUDE    1794
ELEVATION    1794
DATE         1794
AWND         1781
PGTM            1
PRCP         1794
SNOW         1793
SNWD         1793
TAVG            0
TMAX         1794
TMIN         1794
WSF2         1784
WSF5         1768
WT01          508
WT02           32
WT04            5
WT06            7
WT08          347
WT13           27
WT14            1
WT16           23
WT18           21
WT19            9
WT22            2
dtype: int64

In [12]:
df2.head(1)

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PGTM,PRCP,SNOW,...,WT02,WT04,WT06,WT08,WT13,WT14,WT16,WT18,WT19,WT22
0,USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925,42.7,2013-01-01,6.93,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df2.dtypes

STATION       object
NAME          object
LATITUDE     float64
LONGITUDE    float64
ELEVATION    float64
DATE          object
AWND         float64
PGTM         float64
PRCP         float64
SNOW         float64
SNWD         float64
TAVG         float64
TMAX           int64
TMIN           int64
WSF2         float64
WSF5         float64
WT01         float64
WT02         float64
WT04         float64
WT06         float64
WT08         float64
WT13         float64
WT14         float64
WT16         float64
WT18         float64
WT19         float64
WT22         float64
dtype: object

In [18]:
df2.STATION.unique()

array(['USW00094728'], dtype=object)

In [ ]:
df2.columns

#### Storm data

In [15]:
dfbad = pd.read_csv('storm_data_search_results.csv')
dfbad.head()

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,BEGIN_RANGE,BEGIN_AZIMUTH,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,ABSOLUTE_ROWNUMBER
0,421883,NEW YORK (MANHATTAN) (ZONE),,01/31/2013,0,High Wind,50.00,,0,0,...,,,,,,,,,,1
1,436900,NEW YORK (MANHATTAN) (ZONE),,02/08/2013,1400,Winter Storm,,,0,0,...,,,,,,,,,,2
2,452589,NEW YORK CO.,NEW YORK,05/11/2013,1508,Thunderstorm Wind,61.00,,0,0,...,1,SW,1,SW,NEW YORK,40.716,-74.0165,40.716,-74.0165,3
3,452588,NEW YORK CO.,NEW YORK,05/11/2013,1508,Thunderstorm Wind,65.00,,0,0,...,2,SSW,2,SSW,NEW YORK,40.7108,-74.0161,40.7108,-74.0161,4
4,459595,NEW YORK CO.,CENTRAL PARK,06/02/2013,2259,Flash Flood,,,0,0,...,2,NNE,2,NNE,CENTRAL PARK,40.8032,-73.9314,40.801,-73.9321,5


In [16]:
dfbad['EVENT_TYPE'].unique()

array(['High Wind', 'Winter Storm', 'Thunderstorm Wind', 'Flash Flood',
       'Flood', 'Excessive Heat', 'Strong Wind', 'Dense Fog',
       'Winter Weather', 'Heavy Snow', 'Blizzard'], dtype=object)

In [17]:
dfbad[dfbad['EVENT_TYPE'] == 'Flood']

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,BEGIN_RANGE,BEGIN_AZIMUTH,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,ABSOLUTE_ROWNUMBER
5,467533,NEW YORK CO.,CENTRAL PARK,06/07/2013,2205,Flood,,,0,0,...,1,NNW,2,NW,CENTRAL PARK,40.7942,-73.9547,40.7939,-73.9732,6
